In [1]:
import json
import requests
import geojson
from geojson2osm import geojson2osm

In [2]:
# https://en.wikipedia.org/wiki/Aix-Marseille-Provence_Metropolis
# https://www.banatic.interieur.gouv.fr/intercommunalite/200054807-metropole-d-aix-marseille-provence

SIREN = "200054807"

In [3]:
# https://geo.api.gouv.fr/decoupage-administratif
url = f"https://geo.api.gouv.fr/epcis/{SIREN}/communes"
geo_api_response = requests.get(url)

if geo_api_response.status_code == 200:
    geo_api_data = geo_api_response.json()
    AMP_COMMUNES = {commune["nom"].strip() for commune in geo_api_data}
    AMP_COMMUNES_CODE = {commune["code"].strip() for commune in geo_api_data}
    print(len(AMP_COMMUNES))
else:
    print("Failed API")
    AMP_COMMUNES = set()
    AMP_COMMUNES_CODE = set()

92


In [5]:
def save_json_structure(input_file, output_file):

    def get_structure(data, indent=0):
        lines = []
        space = " " * indent
        if isinstance(data, dict):
            for key, value in data.items():
                lines.append(f"{space}{key}: {type(value).__name__}")
                lines.extend(get_structure(value, indent + 2))
        elif isinstance(data, list):
            if data:
                lines.append(f"{space}List of {type(data[0]).__name__}")
                for item in data[:3]: 
                    lines.extend(get_structure(item, indent + 2))
            else:
                lines.append(f"{space}Empty list")
        return lines

    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    structure_str = "\n".join(get_structure(data))
    
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(structure_str)
    
    print(f"Output file: {output_file}")

In [6]:
save_json_structure("geojson-file/BouchesDeRhône.geojson", "geojson-file/structure_export.txt")

Output file: geojson-file/structure_export.txt


In [7]:
file_list = ["BouchesDeRhône.geojson", "Var.geojson", "Vacluse.geojson"]
file_list = ["geojson-file/" + file for file in file_list]
merged_features = []

for file_name in file_list:
    with open(file_name, "r", encoding="utf-8") as f:
        data = json.load(f)
        for feature in data.get("features", []):
            props = feature.get("properties", {})
            commune_name = props.get("name", "").strip()
            if commune_name in AMP_COMMUNES:
                merged_features.append(feature)

merged_geojson = {
    "type": "FeatureCollection",
    "features": merged_features
}

with open("geojson-file/AMP_merged.geojson", "w", encoding="utf-8") as f:
    json.dump(merged_geojson, f, ensure_ascii=False, indent=2)

unique_dict = {}
for feature in merged_features:
    props = feature.get("properties", {})
    insee_code = props.get("ref:INSEE", "").strip()
    admin_level = props.get("admin_level", "").strip()
    if insee_code and admin_level == "8":
        if insee_code not in unique_dict:
            unique_dict[insee_code] = feature
        else:
            pass

AMP_data_dedup = {
    "type": "FeatureCollection",
    "features": list(unique_dict.values())
}

with open("geojson-file/AMP_final.geojson", "w", encoding="utf-8") as f:
    json.dump(AMP_data_dedup, f, ensure_ascii=False, indent=2)

print(len(AMP_data_dedup["features"]))

92


In [8]:
# Đọc file GeoJSON
with open("geojson-file/AMP_final.geojson", "r", encoding="utf-8") as f:
    data = geojson.load(f)

# Lọc bỏ những feature có geometry rỗng
valid_features = []
for feature in data.get("features", []):
    geom = feature.get("geometry", {})
    # Kiểm tra xem geometry có chứa "coordinates" và danh sách này không rỗng
    if "coordinates" in geom and geom["coordinates"]:
        valid_features.append(feature)
    else:
        print(f"Loại bỏ feature không hợp lệ: {feature.get('properties', {}).get('name', 'không rõ')}")

# Cập nhật lại dữ liệu GeoJSON với các feature hợp lệ
data["features"] = valid_features

# Thử chuyển đổi
try:
    osm_data = geojson2osm(data)
    with open("geojson-file/AMP_final.osm", "w", encoding="utf-8") as f:
        f.write(osm_data)
    print("Chuyển đổi thành công!")
except Exception as e:
    print("Có lỗi khi chuyển đổi:", e)


Có lỗi khi chuyển đổi: list index out of range
